In [1]:
import sys
import os

# Get the absolute path of parent folder
current_dir = os.path.abspath("")
parent_dir = os.path.join(current_dir, os.pardir)

# Add to sys.path
sys.path.append(parent_dir)

In [2]:
import ml_combat as ml
from ml_combat import data
from ml_combat.MetaModel import MetaModel

In [3]:
import pandas as pd
import numpy as np
import xgboost

In [4]:
import h2o

h2o.init()

from h2o.automl import H2OAutoML


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.6" 2023-01-17; OpenJDK Runtime Environment Temurin-17.0.6+10 (build 17.0.6+10); OpenJDK 64-Bit Server VM Temurin-17.0.6+10 (build 17.0.6+10, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/tmp0y0fcaa_
  JVM stdout: /var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/tmp0y0fcaa_/h2o_jacobworsoe_started_from_python.out
  JVM stderr: /var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/tmp0y0fcaa_/h2o_jacobworsoe_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_jacobworsoe_4bek15
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [6]:

df = ml.data.get_training_flattened()
temp_df = df
temp_df['hour'] = temp_df['ds'].dt.hour
temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)

temp_df['month'] = temp_df['ds'].dt.month
ml.utils.map_month_to_seasonal(temp_df, 'month')


In [7]:
x = ['location', 'ds', 'weather_data_type', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'ceiling_height_agl:m', 'clear_sky_energy_1h:J',
       'clear_sky_rad:W', 'cloud_base_agl:m', 'dew_or_rime:idx',
       'dew_point_2m:K', 'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W',
       'direct_rad_1h:J', 'effective_cloud_cover:p', 'elevation:m',
       'fresh_snow_12h:cm', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',
       'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'is_day:idx',
       'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 'snow_density:kgm3', 'snow_depth:cm',
       'snow_drift:idx', 'snow_melt_10min:mm', 'snow_water:kgm2',
       'sun_azimuth:d', 'sun_elevation:d', 'super_cooled_liquid_water:kgm2',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms', 'hour', 'dayofyear', 'month']
        
m = H2OAutoML(max_models=10, seed=42)
m.train(x=x, y='y', training_frame=h2o.H2OFrame(temp_df))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
13:59:56.984: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),3/10
# GBM base models (used / total),3/6
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [9]:
m.leaderboard

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231101_135956,217.097,47130.9,72.2237,nan,47130.9
GBM_4_AutoML_1_20231101_135956,220.028,48412.5,73.2537,nan,48412.5
StackedEnsemble_BestOfFamily_1_AutoML_1_20231101_135956,220.055,48424.2,73.3185,nan,48424.2
GBM_3_AutoML_1_20231101_135956,222.166,49357.9,75.7989,nan,49357.9
GBM_2_AutoML_1_20231101_135956,224.888,50574.6,77.9279,nan,50574.6
GBM_1_AutoML_1_20231101_135956,231.374,53533.8,78.5211,nan,53533.8
GBM_5_AutoML_1_20231101_135956,235.101,55272.4,82.4549,nan,55272.4
XRT_1_AutoML_1_20231101_135956,245.478,60259.3,81.4096,0.896267,60259.3
DRF_1_AutoML_1_20231101_135956,245.928,60480.7,81.4084,0.850831,60480.7
GBM_grid_1_AutoML_1_20231101_135956_model_1,250.904,62952.8,87.1046,nan,62952.8


In [12]:
best_model = h2o.get_model(m.leaderboard[0, 'model_id'])

In [29]:
class H2OModel(MetaModel):
    
    def __init__(self, model):
        super().__init__("Naive Model")

        self.model = model
        
    def preprocess(self, df):
        """
        """
        temp_df = df.copy()

        temp_df = df
        temp_df['hour'] = temp_df['ds'].dt.hour
        temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

        temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
        temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)

        temp_df['month'] = temp_df['ds'].dt.month
        ml.utils.map_month_to_seasonal(temp_df, 'month')

        temp_df = h2o.H2OFrame(temp_df)

        return temp_df

    def train(self, df):
        """
        """
        df = self.preprocess(df)

    def predict(self, df):

        df = self.preprocess(df)
        
        return self.model.predict(df).as_data_frame().rename(columns={'predict': 'y_pred'})

In [21]:
future = ml.data.get_testing_flattened()

temp_df = future.copy()

temp_df['hour'] = temp_df['ds'].dt.hour
temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2

temp_df['dayofyear'] = temp_df['ds'].dt.day_of_year
temp_df['dayofyear'] = np.sin(2 * np.pi * (temp_df['dayofyear'] - 80)/ 365)

temp_df['month'] = temp_df['ds'].dt.month
ml.utils.map_month_to_seasonal(temp_df, 'month')

temp_df = h2o.H2OFrame(temp_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [23]:
ret = best_model.predict(temp_df)

stackedensemble prediction progress: |

███████████████████████████████████████████| (done) 100%


In [33]:
retcsv = ret.as_data_frame()

In [30]:
lr = H2OModel(best_model)
df = ml.data.get_training_flattened()
lr.test(df)

Testing Naive Model


/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


...

/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
MAE Vals [74.90955875959831, 21.224641179748627, 21.104867615978428, 12.984776637463881, 10.835525266638937]


[74.90955875959831,
 21.224641179748627,
 21.104867615978428,
 12.984776637463881,
 10.835525266638937]

In [31]:

for location in ['A', 'B', 'C']:
    temp_df = df[df['location'] == location]

    lr = H2OModel(best_model)
    lr.test(temp_df)



print("Done creating a linear regression model!")

Testing Naive Model


/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |

/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


...

████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |

/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = temp_df['ds'].dt.hour
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['hour'] = (np.sin(2 * np.pi * (temp_df['hour'] - 4)/ 24) + 1) / 2
/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_22765/2933109003.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
MAE Vals [20.029927174884882, 5.901911749074829, 18.233019521080553, 6.24114244754642, 11.8542405607371]
Done creating a linear regression model!


In [35]:
retcsv
retcsv.predict = retcsv.predict.apply(lambda a : max(a, 0))


In [38]:
retcsv.reset_index().rename(columns={'index': 'id', 'predict': 'prediction'}).to_csv(ml.module_dir + "/../submissions/" + 'automl.csv', index=False)